## 데이터 가져오기 및 분포 칼럼 확인

In [2]:
import pandas as pd
train_x = pd.read_csv("./open (3)/train.csv")
test_x = pd.read_csv("./open (3)/train.csv")
train_x = pd.DataFrame(train_x)
test_x_ID = test_x["ID"]
target_y = train_x["임신 성공 여부"]

In [3]:
numeric_cols = train_x.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = train_x.select_dtypes(include=["object", "category"]).columns
bool_cols = train_x.select_dtypes(include=["bool"]).columns

excluded = numeric_cols.union(categorical_cols).union(bool_cols)
exception_cols = train_x.columns.difference(excluded).tolist()

print("수치형:", numeric_cols)
print("범주형:", categorical_cols)
print("불리언:", bool_cols)
print("이외 나머지 칼럼:", exception_cols)

수치형: Index(['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '총 생성 배아 수', '미세주입된 난자 수',
       '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부',
       '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부',
       '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일',
       '임신 성공 여부'],
      dtype='object')
범주형: Index(['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형',
       '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
   

# 특정 시술 유형 칼럼 결측치 처리

In [4]:
import pandas as pd

def clean_specific_type(text):
    # 결측치가 있으면 Unknown 으로 분류하기
    if pd.isna(text):
        return pd.Series(["Unknown", 0, 0])
    
    # 텍스트 표준화 (대문자 변환, 공백 제거)
    text = str(text).upper().strip()
    
    # 1. 추가 기법 추출 (Binary Feature)
    is_blastocyst = 1 if "BLASTOCYST" in text else 0
    is_ah = 1 if "AH" in text else 0 # AH: 보조부화술
    
    # 2. 주요 시술 유형 단순화 (Main Category)
    if "ICSI" in text and "IVF" in text:
        main_type = "Mixed (IVF+ICSI)" # 혼합된 경우
    elif "ICSI" in text:
        main_type = "ICSI"
    elif "IVF" in text:
        main_type = "IVF"
    elif "IUI" in text:
        main_type = "IUI"
    elif "ICI" in text:
        main_type = "ICI"
    elif "DI" in text:
        main_type = "DI"
    elif "Unknown" in text:
        main_type = "Unknown"
    else:
        main_type = "Other" # 그 외 (GIFT, FER 등 희귀 케이스)

    return pd.Series([main_type, is_blastocyst, is_ah])

new_cols_train = train_x['특정 시술 유형'].apply(clean_specific_type)
new_cols_test = test_x['특정 시술 유형'].apply(clean_specific_type)
new_cols_train.columns = ['Main_Procedure', 'Is_Blastocyst', 'Is_AH']
new_cols_test.columns = ['Main_Procedure', 'Is_Blastocyst', 'Is_AH']

# 기존 데이터프레임에 합치기
train_x_cleaned = pd.concat([train_x, new_cols_train], axis=1)
test_x_cleaned = pd.concat([test_x, new_cols_test], axis = 1)

# 특정 시술 유형 데이터 칼럼 제거(중복 방지)

In [5]:
train_x = train_x_cleaned.drop("특정 시술 유형", axis = 1)
test_x = test_x_cleaned.drop("특정 시술 유형", axis = 1)

In [6]:
for col in train_x.columns:
    print(f"\n[{col}]")
    print(f"{train_x[col].unique()}")


[ID]
['TRAIN_000000' 'TRAIN_000001' 'TRAIN_000002' ... 'TRAIN_256348'
 'TRAIN_256349' 'TRAIN_256350']

[시술 시기 코드]
['TRZKPL' 'TRYBLT' 'TRVNRY' 'TRJXFG' 'TRXQMD' 'TRCMWS' 'TRDQAZ']

[시술 당시 나이]
['만18-34세' '만45-50세' '만35-37세' '만38-39세' '만40-42세' '만43-44세' '알 수 없음']

[임신 시도 또는 마지막 임신 경과 연수]
[nan 10.  6.  7.  8. 12.  9.  3. 16. 11. 15.  5. 13. 17.  4. 14. 19. 20.
 18.  2.  0.  1.]

[시술 유형]
['IVF' 'DI']

[배란 자극 여부]
[1 0]

[배란 유도 유형]
['기록되지 않은 시행' '알 수 없음' '세트로타이드 (억제제)' '생식선 자극 호르몬']

[단일 배아 이식 여부]
[ 0.  1. nan]

[착상 전 유전 검사 사용 여부]
[nan  1.]

[착상 전 유전 진단 사용 여부]
[ 0. nan  1.]

[남성 주 불임 원인]
[0 1]

[남성 부 불임 원인]
[0 1]

[여성 주 불임 원인]
[0 1]

[여성 부 불임 원인]
[0 1]

[부부 주 불임 원인]
[0 1]

[부부 부 불임 원인]
[0 1]

[불명확 불임 원인]
[0 1]

[불임 원인 - 난관 질환]
[0 1]

[불임 원인 - 남성 요인]
[1 0]

[불임 원인 - 배란 장애]
[1 0]

[불임 원인 - 여성 요인]
[0]

[불임 원인 - 자궁경부 문제]
[0 1]

[불임 원인 - 자궁내막증]
[0 1]

[불임 원인 - 정자 농도]
[0 1]

[불임 원인 - 정자 면역학적 요인]
[0 1]

[불임 원인 - 정자 운동성]
[0 1]

[불임 원인 - 정자 형태]
[0 1]

[배아 생성 주요 이유]
['현재 시술용' nan '난자 저장용' '배아 저장용' '기

In [7]:
train_x.dropna(subset="착상 전 유전 진단 사용 여부", inplace=True)
train_x[numeric_cols].isna().sum()

임신 시도 또는 마지막 임신 경과 연수    240993
배란 자극 여부                      0
단일 배아 이식 여부                   0
착상 전 유전 검사 사용 여부         247342
착상 전 유전 진단 사용 여부              0
남성 주 불임 원인                    0
남성 부 불임 원인                    0
여성 주 불임 원인                    0
여성 부 불임 원인                    0
부부 주 불임 원인                    0
부부 부 불임 원인                    0
불명확 불임 원인                     0
불임 원인 - 난관 질환                 0
불임 원인 - 남성 요인                 0
불임 원인 - 배란 장애                 0
불임 원인 - 여성 요인                 0
불임 원인 - 자궁경부 문제               0
불임 원인 - 자궁내막증                 0
불임 원인 - 정자 농도                 0
불임 원인 - 정자 면역학적 요인            0
불임 원인 - 정자 운동성                0
불임 원인 - 정자 형태                 0
총 생성 배아 수                     0
미세주입된 난자 수                    0
미세주입에서 생성된 배아 수               0
이식된 배아 수                      0
미세주입 배아 이식 수                  0
저장된 배아 수                      0
미세주입 후 저장된 배아 수               0
해동된 배아 수                      0
해동 난자 수                       0
수집된 신선 난

# 배아 생성 주요 이유 칼럼 결측치 처리

In [8]:
import pandas as pd

# 데이터 로드
df_embryo_train = pd.DataFrame(train_x["배아 생성 주요 이유"])
df_embryo_test = pd.DataFrame(test_x["배아 생성 주요 이유"])

def process_embryo_reason_list(text):
    if pd.isna(text):
        return [0, 0, 0, 0, 0]
    
    text = str(text).replace(" ", "").split() # 공백 제거
    
    # 각 키워드가 포함되어 있는지 확인 (포함되면 1, 아니면 0)
    is_current = 1 if "현재시술용" in text else 0
    is_embryo_store = 1 if "배아저장용" in text else 0
    is_egg_store = 1 if "난자저장용" in text else 0
    is_donation = 1 if "기증용" in text else 0
    is_research = 1 if "연구용" in text else 0
    
    return [is_current, is_embryo_store, is_egg_store, is_donation, is_research]

In [9]:
train_reason = train_x['배아 생성 주요 이유'].apply(process_embryo_reason_list)
train_reason = pd.DataFrame(train_reason.tolist(), columns=[
    'Is_Current_Treatment', 
    'Is_Embryo_Storage', 
    'Is_Egg_Storage', 
    'Is_Donation', 
    'Is_Research'
], index=train_x.index)

# test
test_reason = test_x['배아 생성 주요 이유'].apply(process_embryo_reason_list)
test_reason = pd.DataFrame(test_reason.tolist(), columns=[
    'Is_Current_Treatment', 
    'Is_Embryo_Storage', 
    'Is_Egg_Storage', 
    'Is_Donation', 
    'Is_Research'
], index=test_x.index)

# 원본에 붙이기
train_x = pd.concat([train_x.drop("배아 생성 주요 이유", axis = 1), train_reason], axis=1)
test_x  = pd.concat([test_x.drop("배아 생성 주요 이유", axis = 1), test_reason], axis=1)

### 카테고리칼럼을 결측치 유무 확인

In [10]:
categorical_cols = train_x.select_dtypes(include=["object", "category"]).columns
for col in categorical_cols:
    print(train_x[col].isna().sum())
categorical_cols

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Index(['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '배란 유도 유형', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', 'Main_Procedure'],
      dtype='object')

## 나이 칼럼 숫자형으로 변형

In [11]:
age_map = {
    "만18-34세": 1,
    "만35-37세": 2,
    "만38-39세": 3,
    "만40-42세": 4,
    "만43-44세": 5,
    "만45-50세": 6,
}
train_x["시술 당시 나이"] = train_x["시술 당시 나이"].replace(age_map)
test_x["시술 당시 나이"] = test_x["시술 당시 나이"].replace(age_map)

## 횟수형 칼럼들 문자열에서 전부 수치형으로 변형

In [12]:
import pandas as pd
import numpy as np  
# 1. 변환할 컬럼 리스트 정의
target_cols = [
    "임신 시도 "
    "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", 
    "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", 
    "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수"
]

# 2. 매핑 딕셔너리 정의
mapping_dict = {
    '0회': 0,
    '1회': 1,
    '2회': 2,
    '3회': 3,
    '4회': 4,
    '5회': 5,
    '6회 이상': 6
}

# 3. train, test 데이터프레임 모두에 적용
for df in [train_x, test_x]:
    for col in target_cols:
        # 컬럼이 존재하고 문자열(object) 타입인 경우에만 변환 수행
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].map(mapping_dict)

## 불임 원인 종합칼럼

In [13]:
numerical_cols = train_x.select_dtypes(include=["int64", "float64"]).columns
train_x[numerical_cols].columns

Index(['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수',
       'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수',
       'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
       '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일',
       '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '임신 성공 여부', 'Is_Blastocyst',
       'Is_AH', 'Is_Current_Treatment', 'Is

In [14]:
drop_cols = ["불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제"]

train_x.drop(columns=drop_cols, inplace=True, errors="ignore")
test_x.drop(columns=drop_cols, inplace=True, errors="ignore")

## 수치형 데이터 중 nan 있는 값 전처리

In [15]:
cols = ["착상 전 유전 검사 사용 여부", "PGD 시술 여부", "PGS 시술 여부"]

train_x.loc[:, cols] = train_x[cols].fillna(0)
test_x.loc[:, cols]  = test_x[cols].fillna(0)

train_x.drop("난자 채취 경과일", axis = 1, inplace=True)
test_x.drop("난자 채취 경과일", axis = 1, inplace=True)

In [16]:
for c in cols:
    print(f"\n[{c}]")
    print(f"{train_x[c].unique()}")


[착상 전 유전 검사 사용 여부]
[0. 1.]

[PGD 시술 여부]
[0. 1.]

[PGS 시술 여부]
[0. 1.]


In [17]:
cols = ["난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"]
for col in cols:
    print(f"\n[{col}]")
    print(f"{train_x[col].unique()}")


[난자 혼합 경과일]
[ 0. nan  3.  2.  1.  5.  6.  4.  7.]

[배아 이식 경과일]
[ 3. nan  2.  5.  1.  0.  4.  6.  7.]

[배아 해동 경과일]
[nan  0.  5.  3.  2.  6.  1.  4.  7.]


In [21]:
cols = ["임신 시도 또는 마지막 임신 경과 연수", "난자 해동 경과일", "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"]
for col in cols:
    print(f"\n[{col}]")
    print(f"결측치 수: {train_x[col].isna().sum()}")
    print(f"{train_x[col].unique()}")


[임신 시도 또는 마지막 임신 경과 연수]
결측치 수: 240993
[nan 10.  6.  7.  8. 12.  9. 16. 11. 15.  5. 13. 17.  4. 14.  3. 19. 20.
 18.  2.  0.  1.]

[난자 해동 경과일]
결측치 수: 248624
[nan  0.  1.]

[난자 혼합 경과일]
결측치 수: 47444
[ 0. nan  3.  2.  1.  5.  6.  4.  7.]

[배아 이식 경과일]
결측치 수: 37275
[ 3. nan  2.  5.  1.  0.  4.  6.  7.]

[배아 해동 경과일]
결측치 수: 209691
[nan  0.  5.  3.  2.  6.  1.  4.  7.]


In [17]:
cols = ["임신 시도 또는 마지막 임신 경과 연수", "난자 해동 경과일", "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"]

train_x[cols] = train_x[cols].replace(np.nan, 0)
test_x[cols]  = test_x[cols].replace(np.nan, 0)



In [18]:
numeric_cols = train_x.select_dtypes(include = ["number", "int16"]).columns.tolist()
train_x[numeric_cols].isna().sum()[train_x[numeric_cols].isna().sum()>0]

Series([], dtype: int64)

## 모델 학습 전

In [22]:
# 타깃 분리
y = train_x["임신 성공 여부"].copy()

#학습/테스트 피처 확정
drop_cols = ["ID", "임신 성공 여부"]

X = train_x.drop(columns=drop_cols).copy()
X_test = test_x.drop(columns=["ID"]).copy()

print("X:", X.shape, "| y:", y.shape, "| X_test:", X_test.shape)

# 범주형 컬럼 목록(문자형/카테고리형)
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

#범주형 결측치 처리 (CatBoost NaN 에러 방지)
for c in cat_cols:
    X[c] = X[c].astype("object").fillna("Unknown")
    X_test[c] = X_test[c].astype("object").fillna("Unknown")

cat_idx = [X.columns.get_loc(c) for c in cat_cols]

print("categorical cols:", len(cat_cols))
print("cat cols:", cat_cols[:8])

X: (250060, 70) | y: (250060,) | X_test: (256351, 71)
categorical cols: 10
cat cols: ['시술 시기 코드', '시술 당시 나이', '시술 유형', '배란 유도 유형', '총 시술 횟수', '난자 출처', '정자 출처', '난자 기증자 나이']


## Cetboost

In [23]:
from catboost import CatBoostClassifier

cat_params = dict(
    loss_function="Logloss",
    eval_metric="AUC",

    iterations=6000,          # early stopping
    learning_rate=0.03,       
    depth=6,                  
    l2_leaf_reg=6,            
    random_strength=1.0,      
    bagging_temperature=0.7,  
    
    # 불균형 보정(가볍게)
    class_weights=[1.0, 2.0], 

    # 학습 편의
    verbose=300,
    random_seed=42,
    allow_writing_files=False # 로컬에 catboost_info 폴더 덜 생기게
)

## Lightgbm

In [24]:
import lightgbm as lgb

lgb_params = {

    # 학습 목표
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",

    # 학습 조절
    "learning_rate": 0.03,
    "n_estimators": 10000,      # early stopping
    "num_leaves": 64,           
    "min_child_samples": 50,    

    # 샘플링(과적합 완화 + 종종 성능 향상)
    "subsample": 0.8,
    "subsample_freq": 1,
    "colsample_bytree": 0.8,

    # 규제
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,

    # 기타
    "random_state": 42,
    "n_jobs": -1
}

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
# CV 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 점수 기록
cb_fold_scores = []
lgb_fold_scores = []

In [26]:
# OOF / 테스트 예측 저장

oof_cb = np.zeros(len(X))
test_pred_cb = np.zeros(len(X_test))

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = CatBoostClassifier(**cat_params)

    model.fit(
        X_tr, y_tr,
        cat_features=cat_idx,
        eval_set=(X_va, y_va),
        use_best_model=True,
        early_stopping_rounds=300
    )

    oof_cb[va_idx] = model.predict_proba(X_va)[:, 1]
    auc = roc_auc_score(y_va, oof_cb[va_idx])
    cb_fold_scores.append(auc)

    print(f"[CatBoost] Fold {fold} AUC: {auc:.5f}")

    test_pred_cb += model.predict_proba(X_test)[:, 1] / skf.n_splits

print("CatBoost OOF AUC:", roc_auc_score(y, oof_cb))
print("CatBoost Folds:", cb_fold_scores)

0:	test: 0.7135387	best: 0.7135387 (0)	total: 143ms	remaining: 14m 17s
300:	test: 0.7355546	best: 0.7355546 (300)	total: 19.2s	remaining: 6m 3s
600:	test: 0.7369154	best: 0.7369154 (600)	total: 39.5s	remaining: 5m 55s
900:	test: 0.7372397	best: 0.7372397 (900)	total: 1m 2s	remaining: 5m 51s
1200:	test: 0.7373775	best: 0.7373975 (1087)	total: 1m 24s	remaining: 5m 36s
1500:	test: 0.7374813	best: 0.7374819 (1498)	total: 1m 45s	remaining: 5m 17s
1800:	test: 0.7374646	best: 0.7374896 (1621)	total: 2m 7s	remaining: 4m 57s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7374895892
bestIteration = 1621

Shrink model to first 1622 iterations.
[CatBoost] Fold 1 AUC: 0.73749


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=63]="ICSI": Cannot convert 'ICSI' to float

In [ ]:
import lightgbm as lgb

# 컬럼명 공백만 처리 (경고 방지)
X_lgb = X.copy()
X_test_lgb = X_test.copy()
X_lgb.columns = X_lgb.columns.str.replace(" ", "_")
X_test_lgb.columns = X_test_lgb.columns.str.replace(" ", "_")

# 범주형 dtype 맞추기
cat_cols_lgb = X_lgb.select_dtypes(include=["object", "category"]).columns
for c in cat_cols_lgb:
    X_lgb[c] = X_lgb[c].astype("category")
    X_test_lgb[c] = X_test_lgb[c].astype("category")

oof_lgb = np.zeros(len(X_lgb))
test_pred_lgb = np.zeros(len(X_test_lgb))

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_lgb, y), 1):
    X_tr, X_va = X_lgb.iloc[tr_idx], X_lgb.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = lgb.LGBMClassifier(**lgb_params)

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(300)]
    )

    oof_lgb[va_idx] = model.predict_proba(X_va)[:, 1]
    auc = roc_auc_score(y_va, oof_lgb[va_idx])
    lgb_fold_scores.append(auc)

    print(f"[LGBM] Fold {fold} AUC: {auc:.5f}")

    test_pred_lgb += model.predict_proba(X_test_lgb)[:, 1] / skf.n_splits

print("LGBM OOF AUC:", roc_auc_score(y, oof_lgb))
print("LGBM Folds:", lgb_fold_scores)

In [ ]:
print("CatBoost OOF AUC:", roc_auc_score(y, oof_cb))
print("CatBoost Best Fold AUC:", max(cb_fold_scores))

print("LGBM OOF AUC:", roc_auc_score(y, oof_lgb))
print("LGBM Best Fold AUC:", max(lgb_fold_scores))

In [ ]:
import pandas as pd

# 1) 샘플 제출파일 로드
sample_path = "./open (3)/sample_submission.csv"
sub = pd.read_csv(sample_path)

# 2) 타겟 컬럼(샘플의 2번째 컬럼)을 그대로 사용
target_col = sub.columns[1]

# 3) 길이 검증 (불일치면 바로 원인 찾기)
assert len(test_pred_lgb) == len(sub), f"불일치: pred={len(test_pred_lgb)}, sample={len(sub)}"

# 4) 예측값 채우기
sub[target_col] = test_pred_lgb

# 5) 저장 (UTF-8-SIG 권장: 플랫폼/엑셀 호환)
save_path = "submission_LGBM.csv"
sub.to_csv(save_path, index=False, encoding="utf-8-sig")

print("저장 완료:", save_path)
print("제출 컬럼:", list(sub.columns))
print("행 개수:", len(sub))